In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
#ctu_13 = pd.read_csv("../Datasets/ctu-13.labeled.cleaned.csv")
#ctu_13 = pd.read_csv("../Datasets/ctu_13_without_first_5_charc.csv")
ctu_13 = pd.read_csv("../Datasets/ctu_13_without_first_5_charc_and_rand_pos_maxlen25.csv")

In [3]:
ctu_13.head()

,src,dst,port,proto,State,label,class,modelsize
0,147.32.84.134,147.32.192.34,993.0,tcp,I*z*Z0I0,Normal-Jist,normal,13
1,147.32.84.134,147.32.80.9,53.0,udp,.R.u.u.u.r.u.r.u.U.R.R.u.,Normal-Jist,normal,4879
2,147.32.84.134,173.194.16.96,80.0,tcp,NaN,Normal-Jist,normal,5
3,147.32.84.134,193.41.230.73,80.0,tcp,.Z.Z.I.Z.Z.Z.Z.Z+W.F.Z.Z.,Normal-Jist,normal,41
4,147.32.84.134,193.41.230.85,443.0,tcp,z.Z.I.Z+,Normal-Jist,normal,13


In [4]:
def change_class(x):
    if x == "botnet":
        return 1
    elif x== "normal":
        return 0

ctu_13["class"] = ctu_13["class"].apply(change_class)

Feature Vector

In [5]:
#Periodicity
ctu_13["strong_p"] = ctu_13["State"].str.count('[a-i]')
ctu_13["weak_p"] = ctu_13["State"].str.count('[A-I]')
ctu_13["weak_np"] = ctu_13["State"].str.count('[r-z]')
ctu_13["strong_np"] = ctu_13["State"].str.count('[R-Z]')
#Duration
ctu_13["duration_s"] = ctu_13["State"].str.count('(a|A|r|R|1|d|D|u|U|4|g|G|x|X|7)')
ctu_13["duration_m"] = ctu_13["State"].str.count('(b|B|s|S|2|e|E|v|V|5|h|H|y|Y|8)')
ctu_13["duration_l"] = ctu_13["State"].str.count('(c|C|t|T|3|f|F|w|W|6|i|I|z|Z|9)')
#Size
ctu_13["size_s"] = ctu_13["State"].str.count('[a-c]') + \
                    ctu_13["State"].str.count('[A-C]') + \
                    ctu_13["State"].str.count('[r-t]') + \
                    ctu_13["State"].str.count('[R-T]') + \
                    ctu_13["State"].str.count('[1-3]')
ctu_13["size_m"] = ctu_13["State"].str.count('[d-f]') + \
                    ctu_13["State"].str.count('[D-F]') + \
                    ctu_13["State"].str.count('[u-w]') + \
                    ctu_13["State"].str.count('[U-W]') + \
                    ctu_13["State"].str.count('[4-6]')
ctu_13["size_l"] = ctu_13["State"].str.count('[g-i]') + \
                    ctu_13["State"].str.count('[G-I]') + \
                    ctu_13["State"].str.count('[x-z]') + \
                    ctu_13["State"].str.count('[X-Z]') + \
                    ctu_13["State"].str.count('[7-9]')

In [6]:
#Periodicity %
ctu_13["strong_p"] = ctu_13["strong_p"]/ctu_13["modelsize"]
ctu_13["weak_p"] = ctu_13["weak_p"]/ctu_13["modelsize"]
ctu_13["strong_np"] = ctu_13["strong_np"]/ctu_13["modelsize"]
ctu_13["weak_np"] = ctu_13["weak_np"]/ctu_13["modelsize"]
#Duration %
ctu_13["duration_s"] = ctu_13["duration_s"]/ctu_13["modelsize"]
ctu_13["duration_m"] = ctu_13["duration_m"]/ctu_13["modelsize"]
ctu_13["duration_l"] = ctu_13["duration_l"]/ctu_13["modelsize"]
#Size %
ctu_13["size_s"] = ctu_13["size_s"]/ctu_13["modelsize"]
ctu_13["size_m"] = ctu_13["size_m"]/ctu_13["modelsize"]
ctu_13["size_l"] = ctu_13["size_l"]/ctu_13["modelsize"]

In [7]:
ctu_13.head()

,src,dst,port,proto,State,label,class,modelsize,strong_p,weak_p,weak_np,strong_np,duration_s,duration_m,duration_l,size_s,size_m,size_l
0,147.32.84.134,147.32.192.34,993.0,tcp,I*z*Z0I0,Normal-Jist,0,13,0.0,0.153846,0.076923,0.076923,0.00000,0.0,0.307692,0.000000,0.000000,0.307692
1,147.32.84.134,147.32.80.9,53.0,udp,.R.u.u.u.r.u.r.u.U.R.R.u.,Normal-Jist,0,4879,0.0,0.000000,0.001640,0.000820,0.00246,0.0,0.000000,0.001025,0.001435,0.000000
2,147.32.84.134,173.194.16.96,80.0,tcp,NaN,Normal-Jist,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,147.32.84.134,193.41.230.73,80.0,tcp,.Z.Z.I.Z.Z.Z.Z.Z+W.F.Z.Z.,Normal-Jist,0,41,0.0,0.048780,0.000000,0.243902,0.00000,0.0,0.292683,0.000000,0.048780,0.243902
4,147.32.84.134,193.41.230.85,443.0,tcp,z.Z.I.Z+,Normal-Jist,0,13,0.0,0.076923,0.076923,0.153846,0.00000,0.0,0.307692,0.000000,0.000000,0.307692


In [8]:
ctu_13.drop(["src","dst","port","proto","label","modelsize","State"], axis=1, inplace=True)

In [9]:
ctu_13.dropna(inplace=True)

In [10]:
ctu_13.to_csv("../Datasets/feature_vector_ctu_13.csv", index=False)